# Sentiment Analysis of IMDb Reviews


This notebook implements a sentiment analysis task using IMDb movie reviews. We compare the effectiveness of static embeddings (Word2Vec) and contextual embeddings (BERT) in classifying sentiments expressed in movie reviews.

### 1. Setup and Installation

First, we import necessary libraries and set up our environment.

In [ ]:
%pip install numpy pandas torch transformers scikit-learn matplotlib gensim

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import gensim.downloader as api

### 2. Problem Definition and Hypothesis
#### Problem Definition
The objective of this project is to classify sentiments of IMDb movie reviews using vector-based text representations. Specifically, we aim to determine the efficacy of different embeddings and model architectures (static vs. contextual) in sentiment classification.

#### Hypothesis
We hypothesize that contextual embeddings (like BERT) will perform significantly better than static embeddings (like Word2Vec) for sentiment classification in movie reviews.

### 3. Data Acquisition and Preprocessing
We load the IMDb dataset and preprocess the text by removing HTML tags, converting to lowercase, and removing punctuation.

In [ ]:
# Load the IMDb movie reviews dataset
df = pd.read_csv('C:\Users\serem\Documents\Workspaces\Sentiment Analysis\Data\IMDB Dataset.csv')
df['review'] = df['review'].apply(lambda x: re.sub('<[^>]*>', '', x).lower().replace('[^\w\s]', ' ').strip())

# Split data into training and testing sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['review'], df['sentiment'].map({'positive': 1, 'negative': 0}), test_size=0.2, random_state=42)

# Tokenization for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_for_bert(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

train_encodings = tokenize_for_bert(train_texts.tolist())
val_encodings = tokenize_for_bert(val_texts.tolist())

### 4. Model Configuration
#### Static Embeddings with Word2Vec
We utilize pre-trained Word2Vec embeddings to transform text data into vectors and feed these into a simple neural network for classification.

In [ ]:
# Load pre-trained Word2Vec
word_vectors = api.load("word2vec-google-news-300")

# Model for Word2Vec embeddings
class SentimentClassifier(nn.Module):
    def __init__(self):
        super(SentimentClassifier, self).__init__()
        self.fc1 = nn.Linear(300, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.sigmoid(x)

static_model = SentimentClassifier().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

#### Contextual Embeddings with BERT
We employ a pre-trained BERT model from the Hugging Face library, fine-tuning it on the sentiment classification task.

In [ ]:
# Load BERT model
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

### 5. Experimentation
We perform training and validation for both models.

#### Define a Dataloader

In [ ]:
# Define a dataloader
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = IMDbDataset(train_encodings, train_labels.tolist())
val_dataset = IMDbDataset(val_encodings, val_labels.tolist())
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

#### Training and Evaluation Functions

In [ ]:
# Training function
def train(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(data_loader)

#### 6. Model Training
Train the BERT model and validate its performance.

In [ ]:
# Optimizers
bert_optimizer = AdamW(bert_model.parameters(), lr=2e-5)
static_optimizer = optim.Adam(static_model.parameters(), lr=0.001)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(3):
    train_loss = train(bert_model, train_loader, bert_optimizer, device)
    val_loss = evaluate(bert_model, val_loader, device)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss}, Val Loss: {val_loss}")

### 7. Evaluation
Assess the performance of the BERT model using accuracy and other metrics.

In [ ]:
def get_predictions(model, data_loader, device):
    model.eval()
    predictions = []
    real_values = []
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(inputs, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds)
            real_values.extend(labels)
    return predictions, real_values

# Get predictions
predictions, real_values = get_predictions(bert_model, val_loader, device)
accuracy = accuracy_score(real_values, predictions)
precision = precision_score(real_values, predictions, average='binary')
recall = recall_score(real_values, predictions, average='binary')
f1 = f1_score(real_values, predictions, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

### 8. Visualization
Visualize the performance metrics using histograms.

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(predictions, bins=2, alpha=0.7, color='blue', label='Predictions')
plt.hist(real_values, bins=2, alpha=0.7, color='red', label='Actual')
plt.xlabel('Sentiments')
plt.ylabel('Number of Samples')
plt.legend()
plt.show()

### 9. Conclusions and Future Work
#### Findings
The BERT model demonstrates strong performance in sentiment classification, with high accuracy, precision, recall, and F1 scores.
The performance of the BERT model suggests that contextual embeddings capture the nuances of sentiment better than static embeddings.
#### Limitations
The training time and computational resources required for BERT are significantly higher than those for Word2Vec-based models.
The current implementation does not include hyperparameter tuning, which could potentially improve model performance.
#### Future Directions
Implement and evaluate the Word2Vec-based model for a direct comparison with BERT.
Experiment with other contextual models like RoBERTa or GPT to see if they offer performance improvements.
Explore different preprocessing techniques and their impact on model performance.
Conduct hyperparameter tuning to optimize model performance further.